This script set up the input of the O3 box model with the source apportionment technique using PMF-solved results 

In [1]:
import pandas as pd
import numpy as np
from os import listdir
from os.path import join, isfile

The precentage of concentrations (ppbv) of VOCs used in PMF analysis to the total concentration represented in the box model 

In [9]:
## get summed up box model input concentration (total concentration from measurement)
folderpath = '../../data/raw/Model_input_nosource/'
filelist = [f for f in listdir(folderpath) if 
            isfile(join(folderpath, f))]

# create df to store all daily data
df_merge = pd.DataFrame(columns=['No.','Name','Value','Date'])
for i in range(len(filelist)):
 
    # get input file at previous version
    filepath = folderpath + filelist[i] 
    df_daily = pd.read_csv(filepath, skiprows=3,nrows=680,delim_whitespace=True,header=None)
    df_daily.columns=['No.','Name','Value']
    df_daily['Date']=filelist[i][13:21]
    
    # merge every daily data into one df
    df_merge = pd.concat([df_merge, df_daily])
    
# sum up VOC concentration by SAPRC name among all days
df_model_sum = df_merge.groupby('No.').agg({'Name':'first','Value':'sum'}).reset_index()


## get pmf results
profile_conc = pd.read_csv('../../data/final/pmf_profile_conc.csv')
contribution = pd.read_csv('../../data/final/pmf_profile_perc_spec.csv')

# get PMF chemical species with its SAPRC group
df_VOC_saprc = pd.read_csv('../../data/raw/PMF_species_saprc.csv')

# get PMF species concentrations 
df_VOC_PMF = pd.read_csv('../../data/raw/PMF_input_conc.csv')
df_VOC_PMF = df_VOC_PMF.melt(id_vars=['Date'], value_vars=df_VOC_PMF.iloc[:, 1:].columns, 
                var_name="Parameter", value_name='Value')

# get PMF concentration in SAPRC group
df_PMF_sum = df_VOC_PMF.merge(df_VOC_saprc[['Parameter','SAPRC']], on='Parameter', how='left').groupby('SAPRC').agg({'Value':'sum'}).reset_index()

In [41]:
# how much VOC used in PMF represented in total VOC in box model
df_VOC_PMF_in_model = df_PMF_sum.merge(df_model_sum, left_on='SAPRC', right_on='Name', how='left')
df_VOC_PMF_in_model.columns = ['SAPRC','PMF', 'No.','Name','Model']
df_VOC_PMF_in_model['Prec'] = df_VOC_PMF_in_model['PMF']/df_VOC_PMF_in_model['Model']/1000
df_VOC_PMF_in_model.loc[df_VOC_PMF_in_model['Prec']>1,'Prec']=1
df_VOC_PMF_in_model

,SAPRC,PMF,No.,Name,Model,Prec
0,ALK2,45.853995,88.0,ALK2,0.062612,0.732352
1,ALK3,80.121764,89.0,ALK3,0.070922,1.000000
2,ALK4,94.860994,90.0,ALK4,0.136987,0.692483
3,ARO1,50.912458,28.0,ARO1,0.072873,0.698644
4,ARO2,18.422343,24.0,ARO2,0.079943,0.230443
5,BALD,18.055570,77.0,BALD,0.024205,0.745952
6,BENZENE,30.215805,32.0,BENZENE,0.028599,1.000000
7,CCHO,226.364890,59.0,CCHO,0.197982,1.000000
8,ETOH,648.435000,82.0,ETOH,0.550670,1.000000
9,HCHO,400.100136,46.0,HCHO,0.355185,1.000000


Calculate the PMF profile use SAPRC species instead of VOC species

In [47]:
# get PMF profile in SAPRC species
df_profile_SAPRC = profile_conc.merge(df_VOC_saprc[['Parameter','SAPRC']], on='Parameter', how='left')
df_profile_SAPRC = df_profile_SAPRC[~(df_profile_SAPRC['SAPRC']=='NROG')]
df_profile_SAPRC = df_profile_SAPRC.groupby('SAPRC').sum().reset_index()

# get the precentage of a SAPRC species in each PMF factor to total concentration of this species 
sum_profile_SAPRC = df_profile_SAPRC.iloc[:,1:].sum(axis=1)
for i in df_profile_SAPRC.iloc[:,1:].index:
    for j in df_profile_SAPRC.iloc[:,1:].columns:
        df_profile_SAPRC.loc[i,j] = df_profile_SAPRC.loc[i,j]/sum_profile_SAPRC[i]

Build box model input with source apportionment

In [48]:
# prepare input file format from .doc file
df = pd.read_csv('../../data/raw/saprc11_rev4_bgX.doc', 
                 skiprows=36, nrows=1183-36, header=None, delim_whitespace=True)

df.columns = ['NO.', 'Name', 'MWT', "C'S", "N'S", 'CONC(0)']
df['No.'] = np.arange(1,1148,1)
df = df[['No.','Name']]

In [54]:
# set up input source apportionment file
folderpath = '../../data/raw/Model_input_nosource/'
filelist = [f for f in listdir(folderpath) if isfile(join(folderpath, f))]

for i in range(len(filelist)):

    # get input file at previous version
    filepath = folderpath + filelist[i] 
    df_daily = pd.read_csv(filepath, skiprows=3,nrows=680,delim_whitespace=True,header=None)
    df_daily.columns=['No.','Name','Value']

    # fill non-tagged species
    nontagged_name = df.iloc[0:517,][df.iloc[0:517,]['Name'].isin(df_daily['Name'])]['Name']
    df_merged = df.merge(df_daily.loc[df_daily['Name'].isin(nontagged_name), 
                                      ['Name','Value']], 
                         on='Name', how='left')

    # initiate tagged species as 0 
    df_merged['Value'] = df_merged['Value'].fillna(0)
    
    # set tagged species concentration for each SAPRC species used in PMF
    for j in df_profile_SAPRC['SAPRC']:
        
        # get total concentration of a SAPRC species
        conc_tot = df_merged.loc[df_merged['Name']==j, 'Value'].iloc[0]
        prec_in_tags = \
            df_VOC_PMF_in_model.loc[df_VOC_PMF_in_model['SAPRC']==j,'Prec'].iloc[0]
        #prec_in_tags = 1
        if j=='TERP':
            prec_in_tags=1
        conc_in_vanilla = conc_tot*(1-prec_in_tags)
        conc_in_tags = conc_tot*prec_in_tags
        
        # set vanilla concentration
        df_merged.loc[df_merged['Name']==j, 'Value'] = conc_in_vanilla
        
        # deal with each tag
        for k in np.arange(1,10,1):
            tag_name = j+'_X'+str(k)
            prec_in_one_tag = df_profile_SAPRC[df_profile_SAPRC['SAPRC']==j].iloc[0,k]
            df_merged.loc[df_merged['Name']==tag_name, 'Value'] = \
                prec_in_one_tag*conc_in_tags
    
    ## writing to txt file 
    # change format
    df_merged['Value'] = df_merged['Value'].apply(lambda x: '%.2e' % x)
    for m in range(df_merged.shape[0]):   
        df_merged['No.'].iloc[m] = "{:>3}".format(df_merged['No.'].iloc[m])
        df_merged['Name'].iloc[m] = "{:<16}".format(df_merged['Name'].iloc[m])
          
    # write to txt file
    newfilepath = '../../data/intermediate/model_input_bgX/' + \
        filelist[i].replace('_cor','_bgX')
    
    with open(filepath, 'r') as f_old:
        contents = f_old.readlines()
    f_old.close()

    with open(newfilepath, 'w') as f:
        dfAsString = df_merged.to_string(header=False, index=False)
        for a in range(3):
            f.write(contents[a])
        f.write(dfAsString)
    f.close()
    
    print('file complete: ',filepath)

c:\Users\Shenglun\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


file complete:  ../../data/raw/Model_input_nosource/redlands_4km_20210710_11_cor.txt


c:\Users\Shenglun\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


file complete:  ../../data/raw/Model_input_nosource/redlands_4km_20210711_11_cor.txt


c:\Users\Shenglun\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


...

file complete:  ../../data/raw/Model_input_nosource/redlands_4km_20211026_11_cor.txt


c:\Users\Shenglun\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


file complete:  ../../data/raw/Model_input_nosource/redlands_4km_20211027_11_cor.txt


c:\Users\Shenglun\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


file complete:  ../../data/raw/Model_input_nosource/redlands_4km_20211028_11_cor.txt
